In [17]:
from cloudvolume import CloudVolume
import numpy as np
import os
import random
import requests
from tifffile import TiffFile
import matplotlib.pyplot as plt
import fsspec
import zarr
import dask.array as da
import pandas as pd
from tqdm import tqdm

In [22]:
IMAGE_DETAILS_URL = "{base}/webclient/imgData/{image_id}/"

IMAGE_ID = 9846137
IDR_BASE_URL = "https://idr.openmicroscopy.org"

qs = {'base': IDR_BASE_URL, 'image_id': IMAGE_ID}
url = IMAGE_DETAILS_URL.format(**qs)
r = requests.get(url)


In [23]:
r.json()

{'id': 9846137,
 'meta': {'imageName': 'Figure_S3B_FIB-SEM_U2OS_20x20x20nm_xy.tif',
  'imageDescription': '',
  'imageAuthor': 'Public data',
  'projectName': 'idr0086-miron-micrographs/experimentD',
  'projectId': 1161,
  'projectDescription': 'Publication Title\nChromatin arranges in chains of mesoscale domains with nanoscale functional topography independent of cohesin\n\nExperiment Description\nElectron Micrographs of the nucleus',
  'datasetName': 'U2OS_chromatin',
  'datasetId': 10740,
  'datasetDescription': '',
  'wellSampleId': '',
  'wellId': '',
  'imageTimestamp': 1594981020.0,
  'imageId': 9846137,
  'pixelsType': 'uint8'},
 'perms': {'canAnnotate': False,
  'canEdit': False,
  'canDelete': False,
  'canLink': False},
 'tiles': False,
 'interpolate': True,
 'size': {'width': 1121, 'height': 775, 'z': 184, 't': 1, 'c': 1},
 'pixel_size': {'x': 0.02, 'y': 0.02, 'z': 0.02},
 'init_zoom': 0,
 'pixel_range': [0, 255],
 'channels': [{'emissionWave': None,
   'label': '0',
   'co

In [24]:
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(flatten_dict(item, f"{new_key}[{i}]", sep=sep).items())
                else:
                    items.append((f"{new_key}[{i}]", item))
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
meta_flat = flatten_dict(r.json())
meta_df = pd.DataFrame(meta_flat.items(), columns=['Field', 'value'])
meta_df.to_csv('IDR/idr_9846137_image_meta.csv', index=False)

In [28]:
print(meta_df.to_markdown(index=False))

| Field                        | value                                                                                                         |
|:-----------------------------|:--------------------------------------------------------------------------------------------------------------|
| id                           | 9846137                                                                                                       |
| meta.imageName               | Figure_S3B_FIB-SEM_U2OS_20x20x20nm_xy.tif                                                                     |
| meta.imageDescription        |                                                                                                               |
| meta.imageAuthor             | Public data                                                                                                   |
| meta.projectName             | idr0086-miron-micrographs/experimentD                                                            